In [35]:
#importing libraries

In [36]:
!pip install lifelines

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import ipywidgets as widgets
from ipywidgets import interactive
import tkinter as tk
from tkinter import ttk
from lifelines import KaplanMeierFitter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [37]:
#reading dataset

In [38]:
df=pd.read_csv("EmployeeAttrition.csv")

In [39]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [40]:
#encoding categorical values

In [41]:
def encode_columns(df, encoding_type='label'):

    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

    if encoding_type == 'label':

        for col in categorical_columns:
            df[col] = pd.factorize(df[col])[0]

    elif encoding_type == 'onehot':

        df = pd.get_dummies(df, columns=categorical_columns)

    return df


In [42]:
df_encoded = encode_columns(df, encoding_type='label')

In [43]:
df_encoded.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,0,0,1102,0,1,2,0,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,1,1,279,1,8,1,0,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,0,0,1373,1,2,2,1,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,1,1,1392,1,3,4,0,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,1,0,591,1,2,1,2,1,7,...,4,80,1,6,3,3,2,2,2,2


In [44]:
#Model prediction and evaluation

In [50]:
from sklearn.model_selection import GridSearchCV

X = df_encoded.drop('Attrition', axis=1)
y = df_encoded['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Define the initial classifiers
classifiers = {
    'SVM': SVC(C=0.2, kernel='poly', gamma='scale'),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, learning_rate=1.0),
    'Gaussian NB': GaussianNB(),
    'Logistic Regression': LogisticRegression(C=1.0, penalty='l2', max_iter=100),
    'K-Nearest Neighbor': KNeighborsClassifier(n_neighbors=5),
    'Extra Tree Classifier': ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0),
    'Decision Tree': DecisionTreeClassifier(criterion='gini', max_depth=None)
}

# Define the hyperparameters to be tuned for each classifier
classifiers_with_hyperparameters = {
    'SVM': (SVC(), {
        'C': [0.1, 1.0, 10.0],
        'kernel': ['linear', 'poly', 'rbf'],
        'gamma': ['scale', 'auto', 0.1, 1.0]
    }),
    'Gradient Boosting': (GradientBoostingClassifier(), {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.1, 0.05, 0.01]
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    }),
    'AdaBoost': (AdaBoostClassifier(), {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.1, 0.5, 1.0]
    }),
    'Gaussian NB': (GaussianNB(), {}),
    'Logistic Regression': (LogisticRegression(solver='lbfgs'), {
        'C': [0.1, 1.0, 10.0],
        'max_iter': [100, 200, 300]
    }),
    'K-Nearest Neighbor': (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    }),
    'Extra Tree Classifier': (ExtraTreesClassifier(), {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    }),
    'Decision Tree': (DecisionTreeClassifier(), {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    })
}

initial_accuracies = {}
final_accuracies = {}

# Loop through classifiers and perform hyperparameter tuning
for name, (classifier, param_grid) in classifiers_with_hyperparameters.items():
    # Train and evaluate the initial classifier
    y_pred_initial = classifiers[name].fit(X_train, y_train).predict(X_test)
    initial_accuracy = accuracy_score(y_test, y_pred_initial)
    initial_accuracies[name] = initial_accuracy

    # Perform hyperparameter tuning
    grid_search = GridSearchCV(classifier, param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Get the best estimator from the grid search
    best_classifier = grid_search.best_estimator_

    # Fit the best classifier on the training data
    best_classifier.fit(X_train, y_train)

    # Evaluate the tuned classifier
    y_pred_tuned = best_classifier.predict(X_test)
    tuned_accuracy = accuracy_score(y_test, y_pred_tuned)
    final_accuracies[name] = tuned_accuracy if tuned_accuracy >= initial_accuracy else initial_accuracy

# Print both the initial and final accuracies
print("Initial Accuracies:")
print(initial_accuracies)
print("\nFinal Accuracies:")
print(final_accuracies)

'''
accuracies_initial = {}
for name, classifier_initial in classifiers_initial.items():
    classifier_initial.fit(X_train, y_train)
    y_pred = classifier_initial.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies_initial[name] = accuracy

plt.figure(figsize=(10, 6))
plt.barh(list(accuracies_initial.keys()), list(accuracies_initial.values()))
plt.xlabel('Accuracy')
plt.title('Accuracy of Different Machine Learning Models')
plt.show()

accuracies = {}
for name, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies[name] = accuracy


plt.figure(figsize=(10, 6))
plt.barh(list(accuracies.keys()), list(accuracies.values()))
plt.xlabel('Accuracy')
plt.title('Accuracy of Different Machine Learning Models')
plt.show()
'''


Initial Accuracies:
{'SVM': 0.8707482993197279, 'Gradient Boosting': 0.8809523809523809, 'Random Forest': 0.8707482993197279, 'AdaBoost': 0.8537414965986394, 'Gaussian NB': 0.8299319727891157, 'Logistic Regression': 0.8877551020408163, 'K-Nearest Neighbor': 0.8707482993197279, 'Extra Tree Classifier': 0.8707482993197279, 'Decision Tree': 0.7687074829931972}

Final Accuracies:
{'SVM': 0.8945578231292517, 'Gradient Boosting': 0.8809523809523809, 'Random Forest': 0.8741496598639455, 'AdaBoost': 0.8537414965986394, 'Gaussian NB': 0.8299319727891157, 'Logistic Regression': 0.8877551020408163, 'K-Nearest Neighbor': 0.8707482993197279, 'Extra Tree Classifier': 0.8707482993197279, 'Decision Tree': 0.7857142857142857}


'\nclassifiers = {\n    \'SVM\': SVC(C=0.2, kernel=\'poly\', gamma=\'scale\'),\n    \'Gradient Boosting\': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1),\n    \'Random Forest\': RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0),\n    \'AdaBoost\': AdaBoostClassifier(n_estimators=50, learning_rate=1.0),\n    \'Gaussian NB\': GaussianNB(),\n    \'Logistic Regression\': LogisticRegression(C=1.0, penalty=\'l2\', max_iter=100),\n    \'K-Nearest Neighbor\': KNeighborsClassifier(n_neighbors=5),\n    \'Extra Tree Classifier\': ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0),\n    \'Decision Tree\': DecisionTreeClassifier(criterion=\'gini\', max_depth=None)\n}\n\n# Define a dictionary of classifiers with their respective hyperparameter grids\nclassifiers_with_hyperparameters = {\n    \'SVM\': (SVC(), {\n        \'C\': [0.1, 1.0, 10.0],\n        \'kernel\': [\'linear\', \'poly\', \'rbf\'],\n

In [ ]:
#Suggestion function

In [ ]:
def attrition_suggestions(attrition_status):
    suggestions = []

    if attrition_status == 'Yes':
        suggestions.append("Create a hybrid working environment, allowing employees to work from home on certain days of the week.")
        suggestions.append("Organize more spare-time events to build solid relationships and foster professional friendships among peers and colleagues.")
        suggestions.append("Distribute travel assignments more evenly, ensuring that each employee has the opportunity to travel without being overloaded.")
        suggestions.append("Improve and clarify incentive and promotion mechanisms to motivate and retain employees.")

    return suggestions


In [ ]:
#checking the function

In [ ]:
predicted_attrition = "Yes"
suggestions = attrition_suggestions(predicted_attrition)


for suggestion in suggestions:
    print(suggestion)
